###Notebook Inicializador - Recarrega todas as databases e tabelas

In [0]:
from pyspark.sql import SparkSession
import os

In [0]:
# Load files from bronze layer

base_path = "/mvp/database/"
schemas = ["bronze_aux", "bronze_imoveis", "silver_aux", "silver_imoveis", "ouro_imoveis"]

for schema in schemas:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")
    
def encontrar_tabelas_delta(schema):
    """
    Recursively search existing delta tables and return their paths
    """
    schema_path = os.path.join(base_path, schema)
    tabelas_delta = []

# List folders in schemas
    try:
        arquivos = dbutils.fs.ls(schema_path)
        for entrada in arquivos:
            if entrada.isDir():
                tabela_path = entrada.path.rstrip("/")

                # Verify if tables has delta logs
                try:
                    dbutils.fs.ls(f"{tabela_path}/_delta_log")
                    nome_tabela = os.path.basename(tabela_path)
                    tabelas_delta.append((schema, nome_tabela, tabela_path))
                except:
                    pass  # not delta tables
    except Exception as e:
        print(f"Error loading {schema_path}: {e}")
    
    return tabelas_delta


# Inicialize all found tables
for schema in schemas:
    tabelas = encontrar_tabelas_delta(schema)
    for schema_name, table_name, path in tabelas:
        full_table_name = f"{schema_name}.{table_name}"
        print(f"Table created: {full_table_name} -> {path}")
        
        # Load and save all tables in dbfs catalog
        spark.read.format("delta").load(path).write.mode("overwrite").saveAsTable(full_table_name)

print("All table were succesfully loaded.")

Table created: bronze_aux.cep_rio -> dbfs:/mvp/database/bronze_aux/cep_rio
Table created: bronze_aux.cep_sp -> dbfs:/mvp/database/bronze_aux/cep_sp
Table created: bronze_imoveis.olx_rio -> dbfs:/mvp/database/bronze_imoveis/olx_rio
Table created: bronze_imoveis.olx_sp -> dbfs:/mvp/database/bronze_imoveis/olx_sp
Table created: bronze_imoveis.vivareal_rio -> dbfs:/mvp/database/bronze_imoveis/vivareal_rio
Table created: bronze_imoveis.vivareal_sp -> dbfs:/mvp/database/bronze_imoveis/vivareal_sp
Table created: bronze_imoveis.zap_rio -> dbfs:/mvp/database/bronze_imoveis/zap_rio
Table created: bronze_imoveis.zap_sp -> dbfs:/mvp/database/bronze_imoveis/zap_sp
Table created: silver_aux.cep_rio -> dbfs:/mvp/database/silver_aux/cep_rio
Table created: silver_aux.cep_sp -> dbfs:/mvp/database/silver_aux/cep_sp
Table created: silver_imoveis.olx_rio -> dbfs:/mvp/database/silver_imoveis/olx_rio
Table created: silver_imoveis.olx_sp -> dbfs:/mvp/database/silver_imoveis/olx_sp


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2358015081565481>:44
     41         print(f"Table created: {full_table_name} -> {path}")
     43         # Load and save all tables in dbfs catalog
---> 44         spark.read.format("delta").load(path).write.mode("overwrite").saveAsTable(full_table_name)
     46 print("All table were succesfully loaded.")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1518 i